# Побудова оцінок та довірчих інтервалів

Будуємо довірчі інтервали для `n = 100, 10_000, 1_000_000`
Рівень довіри = `0,99` (рівень значущості = `0.01`

#### Довірчі інтервали для м.с. та дисперсії у випадку нормально розподіленої генеральної сукупності.
##### Перша частина леми 1. (Пункт А):
Вибіркове математичне сподівання: $\hat{a}_{n}=\frac {1}{n}\sum^{n}_{k=1}X_k$
Вибіркова дисперсія: $\hat{\sigma}^2_{n}=\frac {1}{n}\sum^{n}_{k=1}(X_k-\hat{a}_{n})^2$
Дов інтервал $a \in a_n \pm \frac{z_{\gamma}  \sigma_n}{\sqrt{n}}$
К-ть ітерацій: $n$

In [2]:
import math

import numpy as np
from scipy.stats import chi2, t, norm

In [3]:
def calc_sample_variance(samples, sample_mean: float):
    squared_variance = 0.
    for sample in samples:
        squared_variance += (sample - sample_mean) ** 2

    squared_variance /= (len(samples) - 1)
    return math.sqrt(squared_variance)


def calc_sample_mean(samples):
    mean = 0.
    for sample in samples:
        mean += sample
    return mean / len(samples)


def calc_s_squared(samples, sample_mean):
    s_squared = 0
    for sample in samples:
        s_squared += (sample - sample_mean) ** 2

    s_squared /= (len(samples) - 1)
    return s_squared


# generating samples with help of N(0,1) distribution
def gen_samples(size: int):
    samples = size * [None]
    for i in range(size):
        samples[i] = np.random.normal(0, 1)
    return samples

In [4]:
n_arr = [100, 10_000, 1_000_000]  # for calculating Confidence Intervals
m_arr = [1, 10, 100, 1_000, 10_000]  # for calculating Q_m

samples_list = len(n_arr) * [None]
alfa = 0.01
gama = 1 - alfa
z_gama = 2.575  # norm distro 0.99 quantile

Q_m = lambda q, n: np.sum(q) / n
# calculating variance in integral methods
# незміщена?
calc_variance = lambda q, n, Q: (1 / (n-1)) * (np.sum([qi**2 for qi in q]) - n * Q ** 2)

eta = lambda x: 1 / x - 1
xi_i = lambda x: -1 * np.log(x)

# calculating 0.99 quantiles
print(f"samples_num_ci:{n_arr}\nm_arr:{m_arr}")

samples_num_ci:[100, 10000, 1000000]
m_arr:[1, 10, 100, 1000, 10000]


In [5]:
# calculating Confidence Interval for the EXPECTATION(sigma^2) under the assumption:
# - observed random value ~ N(0,1)
# - sample variance is unknown
def calculate_ci_1(samples):
    sample_mean = calc_sample_mean(samples)
    n_sq_root = math.sqrt(len(samples))
    s_squared = calc_s_squared(samples, sample_mean)

    g = t.ppf((1 + gama) / 2, df=len(samples) - 1)
    return [sample_mean - math.sqrt(s_squared) * g / n_sq_root, sample_mean + math.sqrt(s_squared) * g / n_sq_root]


# calculating Confidence Interval for the EXPECTATION under assumption:
# - observed value distribution is unknown
# asymptotic confidence interval
def calculate_ci_2(samples, gama):
    sample_mean = calc_sample_mean(samples)
    sample_variance = calc_sample_variance(samples, sample_mean)

    n_sq_root = math.sqrt(len(samples))

    c = norm.ppf((1 + gama) / 2)
    return [sample_mean - sample_variance * c / n_sq_root, sample_mean + sample_variance * c / n_sq_root]


# calculating Confidence Interval for the variance under assumption:
# - observed random value ~ N(0,1)
def calculate_ci_3(samples, gama):
    sample_mean = calc_sample_mean(samples)
    g_1 = chi2.ppf((1 - gama) / 2, df=len(samples) - 1)
    g_2 = chi2.ppf((1 + gama) / 2, df=len(samples) - 1)

    squared_difference = 0
    for sample in samples:
        squared_difference = (sample - sample_mean) ** 2

    return [squared_difference / g_2, squared_difference / g_1]

In [6]:

def calculate_2_integral():
    pass

In [7]:
def calculate_3_integral():
    pass

In [8]:
for i, sample_size in enumerate(n_arr):
    samples_list[i] = gen_samples(sample_size)
    sample_mean = calc_sample_mean(samples_list[i])
    sample_variance = calc_sample_variance(samples_list[i], sample_mean)

    ci_1 = calculate_ci_1(samples_list[i])
    print('\n[1] Довірчий інтервал для мат. спод. M при невідомій дисперсії')
    print(f'Кількість спостережень: n = {sample_size}')
    print(f'Вибіркове математичне сподівання: {sample_mean}')
    print(f'Довірчий інтервал:[{ci_1[0]} ; {ci_1[1]}]')
    print(f'Довжина довірчого інтервалу: {ci_1[1] - ci_1[0]}')

    ci_2 = calculate_ci_2(samples_list[i], gama)
    print('\n[2] Довірчий інтервал для мат. спод. M при невідомому розподілі')
    print(f'Кількість спостережень: n = {sample_size}')
    print(f'Вибіркове математичне сподівання: {sample_mean}')
    print(f'Довірчий інтервал:[{ci_2[0]} ; {ci_2[1]}]')
    print(f'Довжина довірчого інтервалу: {ci_2[1] - ci_2[0]}')

    ci_3 = calculate_ci_3(samples_list[i], gama)
    print('\n[3] Lовірчий інтервал для дисперсії')
    print(f'Кількість спостережень: n = {sample_size}')
    print(f'Вибіркова дисперсія = {sample_variance}')
    print(f'Довірчий інтервал:[{ci_3[0]} ; {ci_3[1]}]')
    print(f'Довжина довірчого інтервалу: {ci_3[1] - ci_3[0]}')


[1] Довірчий інтервал для мат. спод. M при невідомій дисперсії
Кількість спостережень: n = 100
Вибіркове математичне сподівання: 0.06884792739108217
Довірчий інтервал:[-0.18979237771574328 ; 0.3274882324979076]
Довжина довірчого інтервалу: 0.5172806102136509

[2] Довірчий інтервал для мат. спод. M при невідомому розподілі
Кількість спостережень: n = 100
Вибіркове математичне сподівання: 0.06884792739108217
Довірчий інтервал:[-0.18481179424607966 ; 0.322507649028244]
Довжина довірчого інтервалу: 0.5073194432743237

[3] Lовірчий інтервал для дисперсії
Кількість спостережень: n = 100
Вибіркова дисперсія = 0.9847691432335096
Довірчий інтервал:[0.0006583494511301198 ; 0.001375758949473151]
Довжина довірчого інтервалу: 0.0007174094983430312

[1] Довірчий інтервал для мат. спод. M при невідомій дисперсії
Кількість спостережень: n = 10000
Вибіркове математичне сподівання: 0.005106193537922027
Довірчий інтервал:[-0.020475835938235286 ; 0.030688223014079342]
Довжина довірчого інтервалу: 0.05116

# Метод 1
Будемо обраховувати інтеграл за допомогою методу Монте-Карло:
$$Q_m = \bf{M}(I(\eta > \xi_{1} + \ldots + \xi_m))$$

(варто підмітити, що із збільшенням m значення інтеграла зменшується у $\sqrt{m}$ разів)

In [9]:
q_1 = lambda eta, samples: 1 if eta > np.sum(samples) else 0
m_arr = [1, 10, 100, 1_000]  # for calculating Q_m
n_0 = 5000  #

for m in m_arr:
    indicators = []

    for j in range(n_0):
        samples = np.random.random(m)
        indicators.append(q_1(eta(np.random.random(1)), samples))

    Q = Q_m(indicators, len(indicators))
    sigma = calc_variance(indicators, len(indicators),  Q)

    print(f"Кількість зразків: {m}")
    print(f"Оцінка Q: {Q}")
    print(f"Вибіркова дисперсія: {sigma}")
    print(f"Асимптотичний довірчий інтервал: {calculate_ci_2(indicators, gama)}")
    print(f"Кількість спостережень: {n_0}")
    print(f"N*: {(2.575 * 100) ** 2 * sigma / (Q ** 2)}\n")

Кількість зразків: 1
Оцінка Q: 0.6816
Вибіркова дисперсія: 0.2170648529705941
Асимптотичний довірчий інтервал: [0.6646282419580315, 0.6985717580419685]
Кількість спостережень: 5000
N*: 30980.24240998434

Кількість зразків: 10
Оцінка Q: 0.1686
Вибіркова дисперсія: 0.14020208041608323
Асимптотичний довірчий інтервал: [0.15496015904191823, 0.18223984095808177]
Кількість спостережень: 5000
N*: 327034.66108405555

Кількість зразків: 100
Оцінка Q: 0.0208
Вибіркова дисперсія: 0.020371434286857372
Асимптотичний довірчий інтервал: [0.015600723894467719, 0.02599927610553228]
Кількість спостережень: 5000
N*: 3122118.654500131

Кількість зразків: 1000
Оцінка Q: 0.002
Вибіркова дисперсія: 0.001996399279855971
Асимптотичний довірчий інтервал: [0.0003723696490788872, 0.003627630350921113]
Кількість спостережень: 5000
N*: 33093437.437487498



In [10]:

m_arr = [1, 10, 100, 1_000, 10_000]  # for calculating Q_m
n_0 = 100
q_2 = lambda samples: 1/(1+np.sum(samples))


for m in m_arr:
    indicators = []
    for j in range(n_0):
        samples = np.random.random(m)
        indicators.append(q_2(samples))
    Q = Q_m(indicators, len(indicators))
    sigma = calc_variance(indicators, len(indicators), Q)
    print(f"Кількість зразків: {m}")
    print(f"Оцінка Q: {Q}")
    print(f"Вибіркова дисперсія: {sigma}")
    print(f"Асимптотичний довірчий інтервал: {calculate_ci_2(indicators, gama)}")
    print(f"Кількість спостережень: {n_0}")
    print(f"N*: {(2.575 * 100) ** 2 * sigma / (Q ** 2)}\n")

Кількість зразків: 1
Оцінка Q: 0.6881338705722736
Вибіркова дисперсія: 0.01961045666683183
Асимптотичний довірчий інтервал: [0.6520626421092399, 0.724205099035307]
Кількість спостережень: 100
N*: 2745.973263961743

Кількість зразків: 10
Оцінка Q: 0.16796231543588333
Вибіркова дисперсія: 0.0007322354311468986
Асимптотичний довірчий інтервал: [0.16099216022006213, 0.17493247065170442]
Кількість спостережень: 100
N*: 1721.002496788734

Кількість зразків: 100
Оцінка Q: 0.019573111540935974
Вибіркова дисперсія: 9.269701548439453e-07
Асимптотичний довірчий інтервал: [0.01932511252622782, 0.01982111055564414]
Кількість спостережень: 100
N*: 160.43550156810085

Кількість зразків: 1000
Оцінка Q: 0.0020039507684895476
Вибіркова дисперсія: 1.5618300845548811e-09
Асимптотичний довірчий інтервал: [0.001993771092109922, 0.002014130444869173]
Кількість спостережень: 100
N*: 25.787791788143654

Кількість зразків: 10000
Оцінка Q: 0.00019992054969760885
Вибіркова дисперсія: 1.2934818760411683e-12
Асимпт

In [ ]:
m_arr = [1, 10, 100, 1_000, 10_000]  # for calculating Q_m
n_0 = 10
